Формулировка лаб4: 
Для датасета с отзывами с маркетплейса на русском языке (https://github.com/sismetanin/rureviews) построить модель для предсказания тональности текста.
1. Использовать как минимум 3 модели машинного обучения, решающие задачу классификации (количество классов определить по анализу датасета)
2. Предсказать тональность при помощи NLP-моделей (deeppavlov, natasha и т.д.) 
3. Определить метрики качества моделей и сравнить полученные результаты, в ячейке markdown представить выводы.

In [1]:
import re
from pathlib import Path

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import pymorphy2
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from deeppavlov import build_model

[nltk_data] Downloading package wordnet to /home/dan005/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Датасет скачан из указанного репозитория, считаем его.

In [2]:
df = pd.read_csv('./women-clothing-accessories.3-class.balanced.csv', sep='\t')

In [3]:
df

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


df.info()

Как видно, строк с NAN'ами нет, это хорошо.

In [4]:
df['sentiment'].value_counts()

negative    30000
neautral    30000
positive    30000
Name: sentiment, dtype: int64

Видим равномерное распределение классов отзывов на 3:
- Отрицательные
- Нейтральные
- Положительные

Сделаем стандартное деление на обучающую и тестовую выборки.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

Считаем русские стоп-слова для дальнейшей обработки:

In [40]:
STOP_WORDS = Path('./stop-ru.txt').read_text().split('\n')
STOP_WORDS[:10]

['а',
 'абсолютно',
 'авторизоваться',
 'активный',
 'алло',
 'алтухов',
 'атмосфера',
 'ах',
 'б',
 'беду']

Выполним токенизацию, лемматизацию и векторизацию.

In [41]:
lemmer = pymorphy2.MorphAnalyzer()
vector = TfidfVectorizer(max_features=5000, stop_words=STOP_WORDS)

In [42]:
def clear_and_lemmatize_series(series: pd.Series) -> pd.Series:
    """Обрабатывает каждое значение `pd.Series` через функцию `_clear_and_lemmatize_text`."""
    return series.apply(_clear_and_lemmatize_text)


def _clear_and_lemmatize_text(text: str) -> str:
    """Оставляет в тексте только буквы и пробелы, переводит в нижний регистр, а затем лемматизирует его."""
    return _clear_text(_lemmatize_text(text))
    

def _clear_text(text: str) -> str:
    """Оставляет в тексте только буквы и пробелы."""
    return re.sub(r'[^\w\s]', ' ', text)


def _lemmatize_text(text: str) -> str:
    """Переводит текст в нижний регистр, а затем лемматизирует его."""
    return ' '.join(lemmer.parse(word)[0].normal_form for word in text.lower().split())

In [43]:
X_train_handled = vector.fit_transform(clear_and_lemmatize_series(X_train))
X_test_handled = vector.transform(clear_and_lemmatize_series(X_test))

/home/dan005/rock_team/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['первых', 'прежнему'] not in stop_words.
  warnings.warn(


Применим три модели классификации:
- Логистическая регрессия
- Случайный лес
- Опорные вектора

In [44]:
models = [
    LogisticRegression(),
    RandomForestClassifier(),
    SVC(),
]

for model in models:
    print(model)

    model.fit(X_train_handled, y_train)
    y_pred = model.predict(X_test_handled)

    print('Accuracy:')
    print(accuracy_score(y_test, y_pred))

    print('Report:')
    print(classification_report(y_test, y_pred))

LogisticRegression()


/home/dan005/rock_team/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy:
0.6996111111111111
Report:
              precision    recall  f1-score   support

    neautral       0.59      0.61      0.60      6060
    negative       0.70      0.68      0.69      5942
    positive       0.81      0.80      0.81      5998

    accuracy                           0.70     18000
   macro avg       0.70      0.70      0.70     18000
weighted avg       0.70      0.70      0.70     18000

RandomForestClassifier()
Accuracy:
0.6712777777777778
Report:
              precision    recall  f1-score   support

    neautral       0.57      0.57      0.57      6060
    negative       0.69      0.66      0.68      5942
    positive       0.75      0.78      0.76      5998

    accuracy                           0.67     18000
   macro avg       0.67      0.67      0.67     18000
weighted avg       0.67      0.67      0.67     18000

SVC()
Accuracy:
0.7072222222222222
Report:
              precision    recall  f1-score   support

    neautral       0.59      0.66      0.

Рассмотрим теперь DeepPavlov:

In [11]:
dp_model = build_model('rusentiment_convers_bert', download=True, install=True)

y_pred = [dp_model([x])[0] for x in X_test]

# Преобразуем все нижеперечисленные столбцы в унифицированное значение 'neautral'.
cols_to_repl = [
    'neutral',
    'skip',
    'speech',
]
y_pred = ['neautral' if x in cols_to_repl else x for x in y_pred]

print('Accuracy:')
print(accuracy_score(y_test, y_pred))

print('Report:')
print(classification_report(y_test, y_pred))


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Ignoring transformers: markers 'python_version < "3.8"' don't match your environment



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
2025-03-15 19:51:11.188 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/classifiers/rusentiment_convers_bert/rusentiment_convers_bert_torch.tar.gz download because of matching hashes
/home/dan005/rock_team/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.Layer

Accuracy:
0.5757222222222222
Report:
              precision    recall  f1-score   support

    neautral       0.44      0.81      0.57      6060
    negative       0.70      0.37      0.48      5942
    positive       0.90      0.54      0.68      5998

    accuracy                           0.58     18000
   macro avg       0.68      0.57      0.58     18000
weighted avg       0.68      0.58      0.58     18000



У нас вышел следующий топ:
- Опорные вектора
- Логистическая регрессия
- Случайный лес
- Модель DeepPavlov "rusentiment_convers_bert"

В работе не использовалось специфичных гиперпараметров для настройки моделей и сравнение происходило на параметрах по умолчанию. При таком подходе выиграла модель опорных векторов, она показала наилучшую точность: 0.7.